# Методы сбора и обработки данных из сети Интернет
## Парсинг HTML. BeautifulSoup, MongoDB

### Задание
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
* Наименование вакансии.
* Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
* Ссылку на саму вакансию.
* Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import tqdm
import pandas as pd
import csv

In [2]:
base_url = 'https://hh.ru'
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36'}

In [3]:
def parse_salary(salary):
    salary = salary.replace(" ", '').replace(' ', '')
    res = [None, None, None]
    
    try:
        match_res = re.match('(\d+).(\d+)(.*)', salary) # dual salary
        res[0] = int(match_res.group(1))
        res[1] = int(match_res.group(2))
        res[2] = match_res.group(3)
    except:
        pass
    
    try:
        match_res = re.match('от(\d+)(.*)', salary) # from-salary
        res[0] = int(match_res.group(1))
        res[2] = match_res.group(2)
    except:
        pass
    
    try:
        match_res = re.match('до(\d+)(.*)', salary) # to-salary
        res[1] = int(match_res.group(1))
        res[2] = match_res.group(2)
    except:
        pass
    
    return res

In [4]:
jobs_list = []
page_number = 41
a = tqdm.tqdm_notebook(total=page_number)

for page in range(page_number):
    params = {'text': 'аналитик', 'page': page}
    response = requests.get(base_url + '/search/vacancy', headers=headers, params=params)
    dom = BeautifulSoup(response.text, 'html.parser')
    
    items = dom.find_all('div', {'class': 'vacancy-serp-item'})
    
    for item in items:
        salary = None
        title = None
        
        title = item.find('a', {'data-qa': 'vacancy-serp__vacancy-title'}).text
        
        try:
            salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).text
            parsed_salary = parse_salary(salary)

        except AttributeError as e:
            parsed_salary = [None, None, None]
        
        link = item.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})['href']
        
        jobs_list.append([title, *parsed_salary, link, base_url])
    
    a.update(1)
    
    if len(dom.find_all('a', {'data-qa': 'pager-next'})) == 0:
        break
    
    

C:\Users\velur\AppData\Local\Temp/ipykernel_14580/2561079726.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  a = tqdm.tqdm_notebook(total=page_number)


  0%|          | 0/41 [00:00<?, ?it/s]

In [6]:
jobs_df = pd.DataFrame(jobs_list, columns=['Наименование вакансии', 'Минимальная зарплата', 'Максимальная зарплата', 'Валюта', 'Ссылка на вакансию', 'Ссылка на сайт'])
jobs_df

,Наименование вакансии,Минимальная зарплата,Максимальная зарплата,Валюта,Ссылка на вакансию,Ссылка на сайт
0,Бизнес-аналитик (аналитика бизнес-процессов),150000.0,230000.0,руб.,https://ekaterinburg.hh.ru/vacancy/52504676?fr...,https://hh.ru
1,Аналитик маркетплейсов,70000.0,200000.0,руб.,https://ekaterinburg.hh.ru/vacancy/52563221?fr...,https://hh.ru
2,"Data Analyst \ Аналитик данных (BI, SQL, Pytho...",170000.0,210000.0,руб.,https://ekaterinburg.hh.ru/vacancy/49881796?fr...,https://hh.ru
3,Ведущий системный аналитик (бизнес-аналитик),200000.0,NaN,руб.,https://ekaterinburg.hh.ru/vacancy/51853107?fr...,https://hh.ru
4,Системный аналитик,NaN,150000.0,руб.,https://ekaterinburg.hh.ru/vacancy/52427322?fr...,https://hh.ru
...,...,...,...,...,...,...
795,Маркетолог-аналитик,35000.0,NaN,руб.,https://ekaterinburg.hh.ru/vacancy/49253630?fr...,https://hh.ru
796,Системный аналитик,NaN,NaN,None,https://ekaterinburg.hh.ru/vacancy/52396797?fr...,https://hh.ru
797,IT Бизнес-аналитик (CRM),NaN,NaN,None,https://ekaterinburg.hh.ru/vacancy/52301456?fr...,https://hh.ru
798,Аналитик,40000.0,NaN,руб.,https://ekaterinburg.hh.ru/vacancy/51944967?fr...,https://hh.ru


In [7]:
jobs_df.to_csv("hh_vacancies.csv", sep=";", encoding='utf-8-sig')